In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning==1.9.4
!pip install einops
!pip install omegaconf
!pip install kornia
!pip install wget

In [ ]:
!nvidia-smi

Thu Apr 13 06:46:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/LDM_training"

from tensorboard import notebook
notebook.list()

In [ ]:
ROOT_PATH = "/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM"

import sys
sys.path.insert(0, ROOT_PATH)

In [ ]:
import torch, wget, os
from zipfile import ZipFile
from omegaconf import OmegaConf
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from modules.util import instantiate_from_config, makeDirectories, trainTestSubdivision, extractFile


def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt)#, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    model.cuda()
    model.eval()
    return model


if __name__ == "__main__":
    # Pytorch and pytorch-lightning settings
    device = "gpu" if torch.cuda.is_available() else "cpu"
    seed_everything(43) # Make the experiment reproducible
    torch.set_float32_matmul_precision('high')

    # Configuration file path and loading
    experiment_cfg_path = ROOT_PATH + "/configs/custom-ldm-cwa-vq-f8.yaml"
    config = OmegaConf.load(experiment_cfg_path)

    # Set all the path we need for the training
    dataset_name = config.data.dataset_name
    model_name = config.model.name

    sample_folder = ROOT_PATH + f"/sample/{model_name}/{dataset_name}"
    checkpts_save_folder = ROOT_PATH + f"/model_checkpts/{model_name}/{dataset_name}"
    logger_path = ROOT_PATH 

    dataset_path = f"/content/Datasets/{dataset_name}"

    vq_gan_pretrained_path = ROOT_PATH + "/pretrained_model/vq-f8"
    pretrained_url = "https://ommer-lab.com/files/latent-diffusion/vq-f8.zip"
    
    # Generate some utility Directories if they does not exixt
    #makeDirectories((sample_folder, checkpts_save_folder, logger_path, vq_gan_pretrained_path))

    # Dataset extraction
    extractFile("/content/drive/MyDrive/Tesi Bardella 2022/Datasets/wikiart/absfig_filtered_tfr.zip", dataset_path)

    # Download Autoencoder pretrained model if does not exist
    if not os.path.isfile(vq_gan_pretrained_path + "/model.ckpt"):
      wget.download(pretrained_url, out = vq_gan_pretrained_path + "/vq-f8.zip")
      with ZipFile(vq_gan_pretrained_path + "/vq-f8.zip", "r") as zipObj:
        zipObj.extractall(vq_gan_pretrained_path)

    config.model.params.first_stage_config.params["ckpt_path"] = vq_gan_pretrained_path + "/model.ckpt"

    # Here we create the data and label files for train and test
    train_path, test_path, train_labels, test_labels = trainTestSubdivision(dataset_path)

    config.data.params.train.params["training_images_list_file"] = train_path
    config.data.params.train.params["train_labels_file"] = train_labels
    config.data.params.validation.params["test_images_list_file"] = train_path
    config.data.params.validation.params["test_labels_file"] = test_labels

    # Batch size and Learning rate setted for the pytorch-lightning Trainer
    bs, base_lr = config.data.params.batch_size, config.model.base_learning_rate
    n_gpus = config.lightning.trainer.n_gpus
    epochs = config.lightning.trainer.n_epochs

    # Model instantiation from the configuration file
    model_ckpt_path = ROOT_PATH + "/LDM_training/version_11/checkpoints/epoch=241-step=56144.ckpt"
    model = instantiate_from_config(config.model)

    if 'accumulate_grad_batches' in config.lightning.trainer:
        accumulate_grad_batches = config.lightning.trainer.accumulate_grad_batches
    else:
        accumulate_grad_batches = 1

    config.lightning.trainer.accumulate_grad_batches = accumulate_grad_batches
    if config.model.scale_lr:
        model.learning_rate = accumulate_grad_batches * n_gpus * bs * base_lr
        print(f"Setting learning rate to {model.learning_rate:.2e} = {accumulate_grad_batches} (accumulate_grad_batches) * {n_gpus} (num_gpus) * {bs} (batchsize) * {base_lr:.2e} (base_lr)")
    else:
        model.learning_rate = base_lr
        print("++++ NOT USING LR SCALING ++++")
        print(f"Setting learning rate to {model.learning_rate:.2e}")


    # Load the data module and the tensorboard logger for the training
    data = instantiate_from_config(config.data)
    logger = TensorBoardLogger(save_dir=logger_path, name="LDM_training")

    trainer_callbacks = instantiate_from_config(config=config.lightning.callbacks.image_logger)

    trainer = pl.Trainer(default_root_dir= checkpts_save_folder,
                         max_epochs=epochs,
                         devices=1, 
                         accelerator=device, 
                         deterministic=True,
                         callbacks= [trainer_callbacks],
                         enable_checkpointing=True, 
                         num_sanity_val_steps = 2,
                         logger=logger, 
                         )
    trainer.fit(model, 
                data, 
                ckpt_path=model_ckpt_path,
                )

INFO:lightning_fabric.utilities.seed:Global seed set to 43


/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/sample/ldm/wikiart directories already exist.
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/model_checkpts/ldm/wikiart directories already exist.
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM directories already exist.
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/pretrained_model/vq-f8 directories already exist.
DONE
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 394.98 M params.
Keeping EMAs of 628.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channe

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/LDM_training/version_11/checkpoints/epoch=241-step=56144.ckpt
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:338: UserWarning: The dirpath has changed from '/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/LDM_training/version_11/checkpoints' to '/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/LDM_training/version_12/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_

LatentDiffusion: Also optimizing conditioner params!


INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/LDM_training/version_11/checkpoints/epoch=241-step=56144.ckpt


...

/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprec

Validation: 0it [00:00, ?it/s]

/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprec

Validation: 0it [00:00, ?it/s]

/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/content/drive/MyDrive/Tesi Bardella 2022/LDM_pipeline_porting/Second_stage_LDM/modules/util.py:239: UserWarning: TypedStorage is deprec